In [1]:
#Importation des modules nécessaires
import numpy as np 
import pandas as pd 
from collections import Counter



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
import seaborn as sns 
import matplotlib.pyplot as plt
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import collections

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, precision_recall_curve

import warnings
warnings.simplefilter('ignore')


# OBSERVATION DES DONNÉES

In [3]:
data_train = pd.read_csv("train.csv")
data_train.head()

ParserError: Error tokenizing data. C error: Expected 3 fields in line 29, saw 4


## CORRÉLATION

In [ ]:
corr = data_train.corr()
plt.figure(figsize = (15,5))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask = mask, robust = True, cmap="viridis")
plt.title('Table de corrélations')
plt.show()

## Analyse des données

On remarque que certains variable tels que V3 et V1 sont moins corréler que d autres, mais dans l'ensemble, il n'y a pas une grande corrélation entre les variables

In [ ]:
data_columns  = list(data_train.columns)
data_description = pd.DataFrame()
for i in data_columns:
    desc = pd.DataFrame(data_train[i].describe())
    data_description.index = desc.index
    data_description[i] = desc[i]
    
data_description.transpose()

1. Les valeurs sont normalisées à l'aide de l'ACP (Analyse en Composantes Principales) et elles se situent dans une plage de valeurs de négatif à positif.
2. Il n'y a pas de valeurs négatives dans les champs montant (Amount), temps (Time) et classe (Class).
3. 75 % des transactions étaient inférieures à 68,9 $.



In [ ]:
class_1 = data_train[data_train['Class']==1]
class_1

In [ ]:
class_1['Time'].describe()

In [ ]:
class_0= data_train[data_train['Class']==0]
class_0

In [ ]:
class_0['Time'].describe()

In [ ]:
class_1['Amount'].describe()

In [ ]:
class_1['Amount'].hist()

In [ ]:
class_0['Amount'].describe()

Analyse des classes 0 et 1
1. Temps - Le temps écoulé pour les deux cas est assez similaire l'un à l'autre. Par conséquent, je ne détecte aucune corrélation entre eux.

2. Montant - En considérant uniquement les transactions frauduleuses, seulement 25 % se situent entre 70 et 4 000 $. Cela indique que le montant volé est très faible.

In [ ]:
data_train['Amount'].plot(kind='hist',bins = 20)

In [ ]:
sns.boxplot(x=data_train['Amount'])

In [ ]:
sns.scatterplot(x = 'V20',y='Amount', data = data_train,hue ='Class')


In [ ]:
sns.scatterplot(x = 'V21',y='Amount', data = data_train,hue ='Class')

Transactions légitimes vs frauduleuses

In [ ]:
data_train['Class'].value_counts()/data_train['Class'].value_counts().sum() * 100

In [ ]:
data_train['Class'].value_counts()

Étant donné que l'ensemble de données présente un fort déséquilibre, nous pouvons créer un échantillon représentatif de l'ensemble de données et commencer à partir de là.

In [ ]:
data_sample = data_train.sample(frac = 1)

data_fraud = data_sample.loc[data_sample['Class']==1]
data_true = data_sample.loc[data_sample['Class']==0][:470]

In [ ]:
data_resampled = pd.concat([data_fraud,data_true])
data_resampled = data_resampled.sample(frac =1,random_state = 51)
data_resampled.reset_index(inplace = True, drop = True)
data_resampled.drop(columns=['id'],inplace = True)
data_resampled

### mise en échelle de Time et amount

In [ ]:
from sklearn.preprocessing import  RobustScaler
from sklearn.model_selection import train_test_split

rc = RobustScaler()
data_resampled['Time'] = rc.fit_transform(data_resampled['Time'].values.reshape(-1,1))
data_resampled['Amount'] = rc.fit_transform(data_resampled['Amount'].values.reshape(-1,1))


In [ ]:
data_resampled

In [ ]:
data_resampled.Class.value_counts()

## COMPARAISON DES MODÈLES

In [ ]:
X = data_resampled.drop('Class', axis=1)
y = data_resampled['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [ ]:
from sklearn.model_selection import cross_val_score

for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Le classifieur: ", classifier.__class__.__name__, "", round(training_score.mean(), 2) * 100, "% accuracy score")

En moyenne SVC et LogisticRegression sont les algorithmes les plus efficaces

In [ ]:
test = pd.read_csv("test.csv")
data_test =test.copy()
data_test.drop(columns=['id'],inplace = True)
data_test['Time'] = rc.fit_transform(data_test['Time'].values.reshape(-1,1))
data_test['Amount'] = rc.fit_transform(data_test['Amount'].values.reshape(-1,1))
data_test

In [ ]:
log_reg = LogisticRegression(solver='liblinear', random_state=0)
log_reg.fit(X_train,y_train)

In [ ]:
y_pred_test = log_reg.predict(X_test)
y_pred_test

In [ ]:
from sklearn.metrics import accuracy_score

print('Score de précision du modèle: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

In [ ]:
print('Score du set de training: {:.4f}'.format(log_reg.score(X_train, y_train)))

print('Score du set de test: {:.4f}'.format(log_reg.score(X_test, y_test)))

PRÉDICTION FINAL

In [ ]:
y_pred_test = log_reg.predict(data_test)
y_pred_test

In [ ]:
log_reg.predict_proba(data_test)

## Rééchantillonnage de l'ensemble de données avec une autre méthode

In [ ]:
data_train['Time'] = rc.fit_transform(data_train['Time'].values.reshape(-1,1))
data_train['Amount'] = rc.fit_transform(data_train['Amount'].values.reshape(-1,1))
data_train

In [ ]:
features = data_train.drop(["id","Class"], axis=1)
labels = data_train["Class"]

X_train1, X_test1, y_train1, y_test1 = train_test_split(features, labels, test_size=0.2, shuffle=True, stratify=labels, random_state=2023)

In [ ]:
print("Dimentions de X_train : {}".format(X_train1.shape))
print("Dimentions de  X_test : {}".format(X_test1.shape))
print("Dimentions de y_train : {}".format(y_train1.shape))
print("Dimentions de  y_test : {}".format(y_test1.shape))

print('Dimensions des Labels y_train :', Counter(y_train1))
print('Dimensions des Labels y_test :', Counter(y_test1))

In [ ]:
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200, random_state=2023)
undersample_X_train, undersample_y_train = undersample.fit_resample(X_train1, y_train1)

In [ ]:
oversample = SMOTE(sampling_strategy=0.7, random_state=2023)
resample_X_train, resample_y_train = oversample.fit_resample(undersample_X_train, undersample_y_train)

In [ ]:
final_model = LogisticRegression(random_state=2023, penalty="l2")
final_model.fit(resample_X_train, resample_y_train)

In [ ]:
y_pred = final_model.predict(X_test1)
y_score = final_model.predict_proba(X_test1)[:,1]

In [ ]:
accuracy = round(accuracy_score(y_test1, y_pred), 4)
recall = round(recall_score(y_test1, y_pred), 4)
precision = round(precision_score(y_test1, y_pred), 4)
f1 = round(f1_score(y_test1, y_pred), 4)
auc_pr = round(average_precision_score(y_test1, y_pred), 4)

In [ ]:
y_pred = final_model.predict(data_test)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['Class'] = y_pred

In [ ]:
print('Score de précision du modèle: {0:0.4f}'. format(accuracy))

In [ ]:
sub.to_csv('submission.csv',index=False)